In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Url'].tolist()

总数量：1374


['https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.7884-20008.html',
 'https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.7884-20009.html',
 'https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.7884-20011.html',
 'https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.7884-20012.html',
 'https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.7884-20013.html',
 'https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.7884-20015.html',
 'https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.7884-20016.html',
 'https://www.gates.com/gb/en/power-transmission/power-transmission-kits/micro-v-kits.p.7884-000000-000000.v.78

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [6]:
a = 0

resp = requests.get(input_.loc[a, 'Url'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'\n<!DOCTYPE HTML>\n<html lang="en">\n    <head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n\t<meta http-equiv="X-UA-Compatible" content="IE=11"/>\n\t<meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no"/>\n    <meta charset="UTF-8"/>\n   \n    <title>Micro-V® Kits | Gates Corporation</title>\n    \n    <meta name="description" content="&lt;p>Save time and energy with the Gates Micro-V&amp;reg; Kit for your accessory belt drive systems. All Gates kits are specifically designed and manufactured as a complete solution for a specific application. You not only save time sourcing parts, but also save money when purchasing a bundle. Get all your accessory belt drive needs in one place, at a great price.&lt;/p>"/>\n    <meta name="languageIsoCode" content="en_gb"/>\n    <meta name="countryIsoCode" content="gb"/>\n    <meta name="google-site-verification" content="YVIbeqrObPJtVvzY7oRoQcGU8He5ZUX7baH_rZyCGZU"/>\n    <meta name="rob

In [7]:
from bs4 import BeautifulSoup
from lxml import etree

In [8]:
soup = BeautifulSoup(resp, 'lxml')
html = etree.HTML(str(soup))

with open('./part.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE HTML>
<html lang="en">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=11" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1, user-scalable=no" name="viewport"/>
  <meta charset="utf-8"/>
  <title>
   Micro-V® Kits | Gates Corporation
  </title>
  <meta content="&lt;p&gt;Save time and energy with the Gates Micro-V&amp;reg; Kit for your accessory belt drive systems. All Gates kits are specifically designed and manufactured as a complete solution for a specific application. You not only save time sourcing parts, but also save money when purchasing a bundle. Get all your accessory belt drive needs in one place, at a great price.&lt;/p&gt;" name="description"/>
  <meta content="en_gb" name="languageIsoCode"/>
  <meta content="gb" name="countryIsoCode"/>
  <meta content="YVIbeqrObPJtVvzY7oRoQcGU8He5ZUX7baH_rZyCGZU" name="google-site-verification"/>
  <meta content="index,follow" name="robots"/>
  

In [9]:
list_tr = html.xpath('//table[@class="table c-table c-table--split"]/tbody/tr')

list_tr

[<Element tr at 0x128d3ef00>,
 <Element tr at 0x1298f2480>,
 <Element tr at 0x1298f2380>,
 <Element tr at 0x128db2a80>]

In [10]:
list_th = [th.strip() for th in html.xpath('//table[@class="table c-table c-table--striped"]/thead/tr/th/text()')]

list_th

['Part #', 'Product #', 'Description', 'Qty']

In [11]:
list_tr_2 = html.xpath('//table[@class="table c-table c-table--striped"]/tbody/tr')

list_tr_2

[<Element tr at 0x129b47cc0>, <Element tr at 0x129b46940>]

In [12]:
df_temp = pd.DataFrame({'序号': int(input_.loc[a, 'No.']),
                        'Url': input_.loc[a, 'Url'],
                        'Kit_序号': [i+1 for i in range(len(list_tr_2))]})

for tr in list_tr:
    df_temp[tr.xpath('./th/text()')[0].strip()] = '' if len(tr.xpath('./td/text()')) == 0 else tr.xpath('./td/text()')[0].strip()

for i in range(len(list_th)):
    list_td = [tr.xpath('./td[' + str(i+1) + ']')[0] for tr in list_tr_2]
    list_td = ['' if td.text == None else td.text.strip() for td in list_td]
    df_temp['Kit_' + list_th[i]] = list_td
    
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

2

In [13]:
crawler_status = 'ok'

crawler_status

'ok'

In [14]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                             'Url': input_.loc[a, 'Url']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [15]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号', 'Kit_序号'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./test_part.xlsx', index=False)

output_correct

,序号,Url,Kit_序号,Part #,Product #,Description,Weight (lbs.),Kit_Part #,Kit_Product #,Kit_Description,Kit_Qty
0,1,https://www.gates.com/gb/en/power-transmission...,1,K015PK880,7884-20008,K015PK880 MICRO V KIT,,5PK880,8653-10106,Gates Micro-V® Belts,1
1,1,https://www.gates.com/gb/en/power-transmission...,2,K015PK880,7884-20008,K015PK880 MICRO V KIT,,T39101,7808-21501,DriveAlign® Automatic Belt Tensioners,1


In [16]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_part_error.xlsx', index=False)

output_error

""
